## Physics-Informed Neural Networks for 1D Burgers Equation

### Introduction

Physics-Informed Neural Networks (PINNs), introduced by *Raissi et al. (2019)*, represent a transformative approach that seamlessly integrates physical laws into the training of neural networks. Unlike traditional neural networks that rely solely on data-driven methodologies, PINNs leverage the underlying governing equations (such as partial differential equations) directly within the loss function of the network.

The central idea behind PINNs is to encode known physical constraints into the neural network's optimization process (basically the loss function), enabling the network not only to fit observational data but also to adhere closely to the underlying physics of the problem. This approach results in neural networks that are inherently consistent with physical laws, making PINNs highly effective in scenarios where data might be sparse or noisy.

PINNs have found applications across various scientific and engineering domains, including fluid dynamics, solid mechanics, quantum mechanics, and more. Their primary advantage is the ability to yield physically accurate predictions with fewer or (no observed) training data points and improved interpretability.

In general, PINNs are unsupervised, but they can be augmented with available data. However, employing PINNs also introduces specific challenges, such as ensuring adequate convergence during training, balancing the contribution of data-fitting (if data is present) and physics-based losses, and managing the computational complexity associated with evaluating the governing equations.

In this notebook, we will demonstrate the practical implementation of PINNs using Pytorch Ligtning.

In [2]:
import os

#Importing the required libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from scipy.io import loadmat
from torch.utils.data import DataLoader, TensorDataset, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import seed_everything
from pathlib import Path

torch.manual_seed(1234)

np.random.seed(1234)

#Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device.type == 'cuda':
  print(torch.cuda.get_device_name(0))

cuda
NVIDIA RTX A6000


### Problem Setup

We consider the Burgers equation in one-dimension, which is a quasi-linear parabolic partial differential equation given as:
\begin{equation}
 \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}, \qquad 0 < x < 1, t > 0,
\end{equation}
with initial condition
$$
    u(x,0) = sin(\pi x), \qquad 0< x < 1,
$$
and (homogenous) boundary conditions  
$$
    u(0,t) =  u(1,t) = 0,  \qquad t > 0.
$$

> Note: The exact solution of the above Burger's equation can be obtained using the Hopf-Cole transformation, that transforms the equation into a linear heat equation.

To solve using PINNs, define a residual $f(t, x)$ as:
\begin{equation}
f := \frac{\partial u_{\theta}}{\partial t} + u_{\theta} \frac{\partial u_{\theta}}{\partial x} - \nu \frac{\partial^2 u_{\theta}}{\partial x^2},
\end{equation}
where $ u_{\theta}(x, t)$ is the velocity in the Burgers equation, now represented as a neural network (i.e is the output of a neural network).

**Our goal is to let $f \approx 0 $, so that our neural network output $u_{\theta}$ will satisfy the Burgers equation (thus, a solution!!).**

What makes a PINN different from a vanilla neural network is basically the loss function. In a PINN, the neural network parameters are trained by minimizing a composite loss function:
$$
MSE = MSE_u + MSE_f,
$$
where
$$
MSE_u = \frac{1}{N_u}\sum_{i=1}^{N_u}\left| u_{\theta}(t_u^i, x_u^i) - u^i \right|^2,
$$
and,
$$
MSE_f = \frac{1}{N_f}\sum_{i=1}^{N_f}\left| f(t_f^i, x_f^i) \right|^2.
$$

In this formulation:

* $ (t_u^i, x_u^i, u^i)$ are observed data points with known values of the solution $ u $, such as the boundary points and/or data from a dataset (or exeperiments).

* $ (t_f^i, x_f^i)$ are collocation points used to enforce the differential equation residual $ f(t,x) \approx 0 $.

> How do we compute the derivatives required for evaluating $ f(t,x) $? They are computed using automatic differentiation (or some known numerical differencing techniques, though this is less common), enabling the simultaneous enforcement of data-fitting and physical constraints.

> In this Notebook, we take $\nu = 1.0$


### Configuration and Data Generation

In [9]:
class config:
  def __init__(self):
    #The training parameters
    self.num_epoch = 300
    self.batch_size = 50
    self.nu = 1.0
    self.x0 = 0  #x0 :Left boundary point
    self.xf = 1  #xf (x final): Right boundary point
    self.layers = [2,125, 256, 125, 1]
    self.spatial_resolution =  300
    self.temporal_resolution = 150
    self.init_cond = lambda x: torch.sin(torch.pi * x) #Defining the initial condition

    #Optimizer
    self.lr = 0.01
    #self.weight_decay = 1e-3  #Regularization weight

    #The learning rate scheduler
    self.step_size = 75  #To decay after every, say 100 epochs
    self.gamma = 0.5      #To reduce the learning rate by gamma (say, 1/2)

    ##Model hyperparameters
    self.hidden_layers = 125 #Hidden layers for trunk and branch



    self.model_path = 'C:/Users/idris_oduola/Documents/Projects/RqPINN/dataset/pinn_burgers_model1d.pt'
    self.checkpoint_dir = 'C:/Users/idris_oduola/Documents/Projects/RqPINN/dataset/checkpoint_burgers1d_pinn'


cfg = config()

In [10]:
#Automatic differentiation in pytorch
def dfx(f,x):
  gouts = torch.ones([x.shape[0],1], dtype=torch.float, device = device)
  return grad([f],[x],grad_outputs=gouts, create_graph=True)[0]

In [ ]:
def prepare_data(cfg):
    x = torch.linspace(cfg.x0,cfg.xf, cfg.spatial_resolution).view(-1,1)
    t = torch.linspace(0,1, cfg.temporal_resolution).view(-1,1)
    #Now we create a mesh to obtain all possible coordinate points 
    x_mesh, t_mesh= torch.meshgrid(x.squeeze(1),t.squeeze(1))
    print(f"Shape of mesh: {x_mesh.shape}, {t_mesh.shape}")

    #Next we transform the mesh into a 2 column vector to obtain the coordinate points that will be passed in the neural network
    x_stack=torch.hstack((x_mesh.transpose(1,0).flatten()[:,None],t_mesh.transpose(1,0).flatten()[:,None]))
    print(f"Shape after stacking: {x_stack.shape}")

    #Extracting the boundaries of the domain
    bound1 = x_stack[0]; bound2 = x_stack[-1]
    print(f"Boundaries of the domain are: {bound1.shape}, {bound2.shape}")

    #Now we define the initial condition
    left_X=torch.hstack((x_mesh[:,0][:,None],t_mesh[:,0][:,None])) # First column # The [:,None] is to give it the right dimension
    left_u=cfg.init_cond(left_X[:,0]).unsqueeze(1)

    #Boundary Conditions
    #Bottom Edge: x = 0, t = [0,1]
    bottom_X=torch.hstack((x_mesh[0,:][:,None],t_mesh[0,:][:,None])) # First row # The [:,None] is to give it the right dimension
    bottom_u=torch.zeros(bottom_X.shape[0],1)

    #The top Edge: x = 1, t =[0,1]
    top_X=torch.hstack((x_mesh[-1,:][:,None],t_mesh[-1,:][:,None])) # Last row # The [:,None] is to give it the right dimension
    top_u=torch.zeros(top_X.shape[0],1)

    #Padding all together to implement the initial and boundary conditions
    X_train=torch.vstack([left_X,bottom_X,top_X])
    Y_train=torch.vstack([left_Y,bottom_Y,top_Y])
    


